In [38]:
import os
import numpy as np
import pandas as pd

In [39]:
folder_path = '../pipeline/sim_lcs_output'

# List to hold individual DataFrames
data_frames = []

# Traverse the directory structure
for root, _, files in os.walk(folder_path):
    for file in files:
        if 'Perm' in file and file.endswith('.csv') and 'ExSTraCS' in file:
            file_path = os.path.join(root, file)
            # Read CSV into a DataFrame
            df = pd.read_csv(file_path, index_col='times')
            # Append to the list
            data_frames.append(df)
    
    # Concatenate all DataFrames
merged_df = pd.concat(data_frames, axis=1, join='outer').reset_index()
merged_df

,b_scores_het_f1000_maf0.2_cens0.1_cv2,b_scores_het_f1000_maf0.2_cens0.1_cv1,b_scores_het_f1000_maf0.2_cens0.1_cv3,b_scores_het_f1000_maf0.2_cens0.1_cv4,b_scores_het_f1000_maf0.2_cens0.1_cv0,b_scores_het_f1000_maf0.2_cens0.8_cv2,b_scores_het_f1000_maf0.2_cens0.8_cv1,b_scores_het_f1000_maf0.2_cens0.8_cv3,b_scores_het_f1000_maf0.2_cens0.8_cv4,b_scores_het_f1000_maf0.2_cens0.8_cv0,...,b_scores_me_f10000_maf0.4_cens0.8_cv2,b_scores_me_f10000_maf0.4_cens0.8_cv1,b_scores_me_f10000_maf0.4_cens0.8_cv3,b_scores_me_f10000_maf0.4_cens0.8_cv4,b_scores_me_f10000_maf0.4_cens0.8_cv0,b_scores_me_f10000_maf0.4_cens0.4_cv2,b_scores_me_f10000_maf0.4_cens0.4_cv1,b_scores_me_f10000_maf0.4_cens0.4_cv3,b_scores_me_f10000_maf0.4_cens0.4_cv4,b_scores_me_f10000_maf0.4_cens0.4_cv0
times,,,,,,,,,,,,,,,,,,,,,
1.0,0.005000,NaN,0.005000,NaN,NaN,0.045644,0.048066,0.051005,0.056779,0.029112,...,0.022571,0.046299,0.043341,0.058204,0.044670,4.564947e-10,NaN,NaN,NaN,5.506738e-07
2.0,0.004999,NaN,0.005000,NaN,NaN,0.089628,0.085264,0.100361,0.077759,0.073133,...,0.061195,0.073387,0.079735,0.095800,0.084769,7.540954e-09,NaN,4.142065e-07,0.010067,3.701084e-06
3.0,0.004998,NaN,0.005000,NaN,NaN,0.121744,0.123049,0.133202,0.104338,0.079308,...,0.120022,0.123272,0.146582,0.165802,0.157861,7.911697e-08,NaN,2.444958e-06,0.020121,1.366767e-05
4.0,0.004998,NaN,0.005000,NaN,NaN,0.142657,0.164783,0.155054,0.131856,0.079292,...,0.175069,0.182022,0.224705,0.257177,0.244094,6.483806e-07,0.010152,1.055633e-05,0.020055,3.716835e-05
5.0,0.004997,NaN,0.010000,NaN,NaN,0.159040,0.198110,0.178867,0.155078,0.117079,...,0.217123,0.236153,0.298170,0.369804,0.320826,3.460890e-06,0.015206,3.638502e-05,0.024989,1.001073e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94.0,0.009085,NaN,0.008689,0.015074,NaN,NaN,6.845554,0.906396,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95.0,NaN,NaN,0.008689,0.007660,NaN,NaN,1.124417,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96.0,NaN,NaN,NaN,0.007660,NaN,NaN,0.693255,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def fun(df, col_name='b_scores_het_f100_maf0.4_cens0.8_perm6_cv1'):
    temp_df = df[[col_name, 'times']]
    temp_df = temp_df.dropna()
    try:
        val = np.trapz(temp_df[col_name], temp_df['times']) / (list(temp_df['times'])[-1] - list(temp_df['times'])[0])
    except Exception as e:
        display(temp_df)
        raise e
    return val

In [ ]:
ibs_df = pd.DataFrame(columns=['Dataset', 'IBS'])
for col in merged_df.columns:
    if 'times' not in col:
        val = fun(merged_df, col)
        ibs_df = pd.concat([pd.DataFrame([[col,val]], columns=ibs_df.columns), ibs_df], ignore_index=True)
ibs_df = ibs_df.sort_values('Dataset').reset_index(drop=True)
ibs_df

In [ ]:
ibs_df[['b', 'scores', 'model', 'nfeat', 'maf', 'cens', 'perm', 'cv']] = pd.DataFrame(list(ibs_df['Dataset'].str.split('_')))
ibs_df.drop('Dataset', axis=1, inplace=True)
ibs_df.drop('scores', axis=1, inplace=True)
ibs_df.drop('b', axis=1, inplace=True)
ibs_df = ibs_df[['model', 'nfeat', 'maf', 'cens', 'perm', 'cv', 'IBS']]
ibs_df

In [ ]:
df = pd.read_csv('../pipeline/ibs_data_all.csv')
df = df.drop('times', axis=1)
df.columns = df.columns.str[9:]
df.columns = ['times'] + list(df.columns[1:])
org_ibs = df

In [ ]:
def calc_perm_pvalue(ibs_df, model, maf, cens):
    temp_df = ibs_df[(ibs_df['model'] == model) & (ibs_df['maf']==maf) & (ibs_df['cens']==cens)]
    data_points = temp_df['IBS'].ravel()
    model_label = model+'_f100_'+maf+'_'+cens
    org_val = fun(org_ibs[[model_label, 'times']], model_label)
    pvalue = sum(data_points < org_val)/len(data_points)
    print(model, maf, cens, org_val, pvalue)
    return pvalue

In [ ]:
models = [ 'me', 'add', 'epi', 'het']
mafs = ['maf0.2', 'maf0.4']
censs = ['cens0.1', 'cens0.4', 'cens0.8']

In [40]:
pval_df = pd.DataFrame(columns=['Models', 'MAF', 'Cens', 'p-val'])
for model in models:
    for maf in mafs:
        for cens in censs:
            pval = calc_perm_pvalue(ibs_df, model, maf, cens)
            pval_df = pd.concat([pd.DataFrame([[model, maf, cens, pval]], columns=pval_df.columns), pval_df], ignore_index=True)
pval_df = pval_df.reindex(index=pval_df.index[::-1]).reset_index(drop=True)
pval_df

In [41]:
pval_df[pval_df['p-val'] <= 0.05]

,times,b_scores_add_f10000_maf0.2_cens0.1,b_scores_add_f10000_maf0.2_cens0.4,b_scores_add_f10000_maf0.2_cens0.8,b_scores_add_f10000_maf0.4_cens0.1,b_scores_add_f10000_maf0.4_cens0.4,b_scores_add_f10000_maf0.4_cens0.8,b_scores_add_f1000_maf0.2_cens0.1,b_scores_add_f1000_maf0.2_cens0.4,b_scores_add_f1000_maf0.2_cens0.8,...,b_scores_me_f1000_maf0.2_cens0.8,b_scores_me_f1000_maf0.4_cens0.1,b_scores_me_f1000_maf0.4_cens0.4,b_scores_me_f1000_maf0.4_cens0.8,b_scores_me_f100_maf0.2_cens0.1,b_scores_me_f100_maf0.2_cens0.4,b_scores_me_f100_maf0.2_cens0.8,b_scores_me_f100_maf0.4_cens0.1,b_scores_me_f100_maf0.4_cens0.4,b_scores_me_f100_maf0.4_cens0.8
0,1.0,0.002503,0.002503,0.074667,0.002503,0.007492,0.000000,0.005000,0.002509,0.147259,...,0.017590,0.155974,0.002503,0.002505,0.030469,0.248015,0.189073,0.053053,0.004999,0.000000
1,2.0,0.002503,0.002503,0.169705,0.002503,0.009972,0.000000,0.005000,0.002509,0.210523,...,0.066999,0.192057,0.001669,0.002514,0.052053,0.253306,0.210771,0.079594,0.004997,0.000000
2,3.0,0.002503,0.002503,0.256846,0.002503,0.005981,0.000000,0.005000,0.005017,0.246846,...,0.086718,0.213288,0.001669,0.002017,0.186911,0.252129,0.258593,0.140990,0.004996,0.000000
3,4.0,0.005006,0.005003,0.339077,0.002503,0.005965,0.000000,0.005000,0.007525,0.291005,...,0.115564,0.224310,0.001669,0.003039,0.198936,0.246592,0.296846,0.152659,0.004995,0.000000
4,5.0,0.005005,0.005003,0.395804,0.002503,0.009946,0.000000,0.005000,0.008781,0.306826,...,0.146164,0.223421,0.001669,0.003047,0.210577,0.238285,0.311036,0.168877,0.006657,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94.0,NaN,0.066380,NaN,0.023403,0.102016,0.781100,0.023594,0.086751,NaN,...,NaN,NaN,0.093237,0.525661,NaN,NaN,NaN,0.010123,0.114648,0.000149
94,95.0,NaN,0.096813,NaN,0.023404,0.102016,0.781102,NaN,0.000315,NaN,...,NaN,NaN,NaN,0.610539,NaN,NaN,NaN,0.010123,0.114859,0.000142
95,96.0,NaN,0.083458,NaN,0.021839,0.102016,0.554650,NaN,0.000289,NaN,...,NaN,NaN,NaN,0.500720,NaN,NaN,NaN,NaN,0.133068,0.000142
96,97.0,NaN,0.111148,NaN,0.017222,0.102016,0.554652,NaN,0.000270,NaN,...,NaN,NaN,NaN,1.454048,NaN,NaN,NaN,NaN,0.198507,NaN
